In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
import os
import logging
from langdetect import detect
from googletrans import Translator, LANGUAGES

# Set up logging configuration
logging.basicConfig(filename='transcription.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Function to convert audio files to WAV format if necessary
def convert_to_wav(audio_file_path):
    file_name, file_extension = os.path.splitext(audio_file_path)
    if file_extension.lower() != ".wav":
        sound = AudioSegment.from_file(audio_file_path)
        wav_file_path = f"{file_name}.wav"
        sound.export(wav_file_path, format="wav")
        return wav_file_path
    return audio_file_path

# Function to recognize speech from an audio file
def transcribe_audio_file(audio_file_path, detect_language=False, show_all=False, save_to_file=False):
    recognizer = sr.Recognizer()
    audio_file_path = convert_to_wav(audio_file_path)

    language = "en-US"  # Default language
    if detect_language:
        with sr.AudioFile(audio_file_path) as source:
            audio_data = recognizer.record(source)
            try:
                temp_text = recognizer.recognize_google(audio_data)
                detected_lang = detect(temp_text)
                language = f"{detected_lang}"
                print(f"Detected language: {language} ({LANGUAGES.get(detected_lang, 'Unknown')})")
            except sr.UnknownValueError:
                print("Language detection failed. Using default language (English).")
            except sr.RequestError as e:
                print(f"Language detection request failed; {e}. Using default language (English).")

    with sr.AudioFile(audio_file_path) as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Recognizing...")
        audio_data = recognizer.record(source)

        try:
            if show_all:
                result = recognizer.recognize_google(audio_data, language=language, show_all=True)
                print(f"Full Result: {result}")
            else:
                result = recognizer.recognize_google(audio_data, language=language)
                print(f"Transcription: {result}")

            if save_to_file:
                output_file = f"{os.path.splitext(audio_file_path)[0]}_transcription.txt"
                with open(output_file, 'w') as file:
                    file.write(str(result))
                print(f"Transcription saved to {output_file}")

            logging.info(f"Transcription successful for {audio_file_path}")
            return result

        except sr.UnknownValueError:
            logging.error("Google Speech Recognition could not understand the audio")
            print("Google Speech Recognition could not understand the audio")
        except sr.RequestError as e:
            logging.error(f"Could not request results from Google Speech Recognition service; {e}")
            print(f"Could not request results from Google Speech Recognition service; {e}")

    return None

# Function to recognize speech from microphone input
def transcribe_from_microphone(detect_language=False, show_all=False, save_to_file=False):
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Please speak into the microphone...")
        audio_data = recognizer.listen(source)

        try:
            result = recognizer.recognize_google(audio_data, show_all=show_all)
            if show_all:
                print(f"Full Result: {result}")
            else:
                print(f"You said: {result}")

            if save_to_file:
                with open("microphone_transcription.txt", 'w') as file:
                    file.write(str(result))
                print("Transcription saved to microphone_transcription.txt")

            logging.info("Microphone transcription successful")
            return result

        except sr.UnknownValueError:
            logging.error("Google Speech Recognition could not understand the audio")
            print("Google Speech Recognition could not understand the audio")
        except sr.RequestError as e:
            logging.error(f"Could not request results from Google Speech Recognition service; {e}")
            print(f"Could not request results from Google Speech Recognition service; {e}")

    return None

# Function to translate text to Hindi
def translate_to_hindi(text):
    translator = Translator()
    try:
        translated = translator.translate(text, src='en', dest='hi')
        print(f"Translated Text: {translated.text}")
        return translated.text
    except Exception as e:
        logging.error(f"Translation failed; {e}")
        print(f"Translation failed; {e}")
        return None

# Main function to demonstrate the program
def main():
    input_method = input("Type 'file' to transcribe an audio file or 'mic' to use the microphone: ").strip().lower()

    if input_method == "file":
        audio_file_path = input("Enter the path to your audio file: ").strip()
        detect_language = input("Detect language automatically? (yes/no): ").strip().lower() == "yes"
        show_all = input("Show all possible transcriptions? (yes/no): ").strip().lower() == "yes"
        save_to_file = input("Save transcription to a file? (yes/no): ").strip().lower() == "yes"
        text = transcribe_audio_file(audio_file_path, detect_language, show_all, save_to_file)

    elif input_method == "mic":
        detect_language = input("Detect language automatically? (yes/no): ").strip().lower() == "yes"
        show_all = input("Show all possible transcriptions? (yes/no): ").strip().lower() == "yes"
        save_to_file = input("Save transcription to a file? (yes/no): ").strip().lower() == "yes"
        text = transcribe_from_microphone(detect_language, show_all, save_to_file)

    else:
        print("Invalid input. Please type 'file' or 'mic'.")
        return

    if text:
        # Translate the text to Hindi
        translate_to_hindi(text if not isinstance(text, dict) else text['alternative'][0]['transcript'])

if __name__ == "__main__":
    main()
